In [90]:
import re
from chembl_webresource_client.new_client import new_client
from typing import List, Dict, Set, Optional, Tuple

def normalize(text: str) -> str:
    """Радикальная нормализация текста"""
    return re.sub(r"[-–—'\",.(){}\[\]:;!?\\/+\s]", "", text.upper())

def basic_clean(text: str) -> str:
    """Мягкая нормализация"""
    return re.sub(r"[-–—'\",.(){}\[\]:;!?\\/]", "", text.upper())

def get_chembl_family_mapping(chembl_ids: List[str]) -> Dict[str, str]:
    """Создаем mapping: любой ChEMBL ID -> ближайший родительский ID из нашего списка"""
    molecule_form = new_client.molecule_form
    mapping = {}
    
    for chembl_id in chembl_ids:
        mapping[chembl_id] = chembl_id  # Сам себе родитель
        
        try:
            # Получаем всех членов семейства
            family_members = list(molecule_form.filter(parent_chembl_id=chembl_id).only('molecule_chembl_id'))
            for member in family_members:
                if 'molecule_chembl_id' in member:
                    mapping[member['molecule_chembl_id']] = chembl_id
        except Exception as e:
            print(f"Ошибка при получении семейства для {chembl_id}: {e}")
    
    return mapping

def get_synonyms_dict(chembl_ids: List[str], family_mapping: Dict[str, str]) -> Dict[str, Set[str]]:
    """Создаем словарь нормализованных синонимов с привязкой к основным ID"""
    molecule = new_client.molecule
    syn_dict = {chembl_id: set() for chembl_id in chembl_ids}
    
    # Собираем все ChEMBL ID, которые относятся к нашим семействам
    all_related_ids = set(family_mapping.keys())
    
    for chembl_id in all_related_ids:
        try:
            mol = molecule.get(chembl_id)
            synonyms = mol.get("molecule_synonyms", [])
            cleaned_synonyms = [normalize(s["molecule_synonym"]) for s in synonyms 
                              if isinstance(s, dict) and "molecule_synonym" in s]
            
            # Привязываем синонимы к основному ID из нашего списка
            main_id = family_mapping[chembl_id]
            syn_dict[main_id].update(cleaned_synonyms)
        except Exception as e:
            print(f"Ошибка при получении синонимов для {chembl_id}: {e}")
    
    return syn_dict

def find_matches(chembl_ids: List[str], drug_names: List[str]) -> Tuple[Dict[str, Dict], List[str]]:
    """Поиск соответствий с привязкой к основным ID из списка"""
    molecule = new_client.molecule
    results = {}
    not_found = set(drug_names)
    
    # Создаем mapping семейств
    family_mapping = get_chembl_family_mapping(chembl_ids)
    all_related_ids = set(family_mapping.keys())
    
    # Создаем словарь синонимов (уже привязанных к основным ID)
    syn_dict = get_synonyms_dict(chembl_ids, family_mapping)
    
    # 1️⃣ Первая стадия: точный поиск по синонимам
    for drug in list(not_found):
        found = False
        for chembl_id in all_related_ids:
            try:
                mol = molecule.get(chembl_id)
                synonyms = mol.get("molecule_synonyms", [])
                for syn in synonyms:
                    if isinstance(syn, dict) and drug.lower() == syn.get("molecule_synonym", "").lower():
                        # Привязываем к основному ID из списка
                        main_id = family_mapping[chembl_id]
                        main_mol = molecule.get(main_id)
                        results[drug] = {
                            "chembl_id": main_id,
                            "preferred_name": main_mol.get("pref_name", "N/A"),
                            "stage": "first",
                            "matched_synonym": syn.get("molecule_synonym"),
                            "original_chembl_id": chembl_id  # Сохраняем оригинальный ID для информации
                        }
                        found = True
                        break
                if found:
                    not_found.remove(drug)
                    break
            except Exception:
                continue
    
    # 2️⃣ Вторая стадия: поиск в верхнем регистре
    for drug in list(not_found):
        upper_drug = drug.upper()
        found = False
        for chembl_id in all_related_ids:
            try:
                mol = molecule.get(chembl_id)
                synonyms = mol.get("molecule_synonyms", [])
                for syn in synonyms:
                    if isinstance(syn, dict) and upper_drug == syn.get("molecule_synonym", "").upper():
                        main_id = family_mapping[chembl_id]
                        main_mol = molecule.get(main_id)
                        results[drug] = {
                            "chembl_id": main_id,
                            "preferred_name": main_mol.get("pref_name", "N/A"),
                            "stage": "second",
                            "matched_synonym": syn.get("molecule_synonym"),
                            "original_chembl_id": chembl_id
                        }
                        found = True
                        break
                if found:
                    not_found.remove(drug)
                    break
            except Exception:
                continue
    
    # 3️⃣ Третья стадия: мягкая нормализация
    for drug in list(not_found):
        cleaned_drug = basic_clean(drug)
        found = False
        for chembl_id in all_related_ids:
            try:
                mol = molecule.get(chembl_id)
                synonyms = mol.get("molecule_synonyms", [])
                for syn in synonyms:
                    if isinstance(syn, dict) and cleaned_drug == basic_clean(syn.get("molecule_synonym", "")):
                        main_id = family_mapping[chembl_id]
                        main_mol = molecule.get(main_id)
                        results[drug] = {
                            "chembl_id": main_id,
                            "preferred_name": main_mol.get("pref_name", "N/A"),
                            "stage": "third",
                            "matched_synonym": syn.get("molecule_synonym"),
                            "original_chembl_id": chembl_id
                        }
                        found = True
                        break
                if found:
                    not_found.remove(drug)
                    break
            except Exception:
                continue
    
    # 4️⃣ Четвёртая стадия: радикальная нормализация
    for drug in list(not_found):
        norm_drug = normalize(drug)
        found = False
        for main_id in chembl_ids:  # Ищем только среди основных ID
            if norm_drug in syn_dict[main_id]:
                try:
                    main_mol = molecule.get(main_id)
                    results[drug] = {
                        "chembl_id": main_id,
                        "preferred_name": main_mol.get("pref_name", "N/A"),
                        "stage": "fourth",
                        "matched_synonym": norm_drug,
                        "original_chembl_id": None  # Не знаем оригинальный ID при таком поиске
                    }
                    not_found.remove(drug)
                    found = True
                    break
                except Exception:
                    continue
    
    return results, list(not_found)

# Пример использования (остаётся таким же)
if __name__ == "__main__":
    chembl_ids = [
        "CHEMBL25", "CHEMBL112", "CHEMBL941", "CHEMBL600", 
        "CHEMBL521", "CHEMBL405"
    ]

    drug_names = [
        "aspirin", "acetyl-salicylic acid", "Acetyl salicylic acid", "AMPHETAMINE ASPARTATE",
        "AMPHETAMINE"
    ]

    found, not_found = find_matches(chembl_ids, drug_names)

    # Вывод результатов
    stages = {
        "first": "✅ Найдено на первой стадии (точное совпадение):",
        "second": "🔄 Найдено на второй стадии (верхний регистр):",
        "third": "🧹 Найдено на третьей стадии (мягкая нормализация):",
        "fourth": "🧨 Найдено на четвёртой стадии (радикальная нормализация):"
    }

    for stage, header in stages.items():
        print(f"\n{header}")
        for drug, info in found.items():
            if info["stage"] == stage:
                original_info = f" (original: {info['original_chembl_id']})" if info['original_chembl_id'] else ""
                print(f"{drug.ljust(25)} -> {info['preferred_name'].ljust(20)} (ChEMBL: {info['chembl_id']}{original_info}, matched: {info.get('matched_synonym', 'N/A')})")

    print("\n❌ Не найдено вовсе:")
    for drug in not_found:
        print(drug)


✅ Найдено на первой стадии (точное совпадение):
AMPHETAMINE ASPARTATE     -> AMPHETAMINE          (ChEMBL: CHEMBL405 (original: CHEMBL1200377), matched: Amphetamine aspartate)
aspirin                   -> ASPIRIN              (ChEMBL: CHEMBL25 (original: CHEMBL25), matched: Aspirin)
AMPHETAMINE               -> AMPHETAMINE          (ChEMBL: CHEMBL405 (original: CHEMBL405), matched: Amphetamine)

🔄 Найдено на второй стадии (верхний регистр):

🧹 Найдено на третьей стадии (мягкая нормализация):
acetyl-salicylic acid     -> ASPIRIN              (ChEMBL: CHEMBL25 (original: CHEMBL25), matched: Acetylsalicylic Acid)

🧨 Найдено на четвёртой стадии (радикальная нормализация):
Acetyl salicylic acid     -> ASPIRIN              (ChEMBL: CHEMBL25, matched: ACETYLSALICYLICACID)

❌ Не найдено вовсе:
